In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance
from tqdm import tqdm
from pathlib import Path
import numpy as np
import pandas as pd
import math

In [3]:
human_slides = ["10835_19 humane BAL Turnbull Blau.svs",
                "10120_19 humane BAL Berliner Blau.svs",
                "10052_19 humane BAL Berliner Blau-001.svs",
                "10080_19 humane BAL Berliner Blau-001.svs",
                "2707_20 TB BAL Human neu.svs",
                "10835_19 humane BAL Berliner Blau.svs",
                "10227_19 humane BAL Berliner Blau.svs",
                "11480_19 humane BAL Berliner Blau.svs",
                "11323_19 humane BAL Berliner Blau.svs",
                "2702_20 BB Human BAL-001.svs",
                "10672_19 humane BAL Berliner Blau.svs",
                "10672_19 humane BAL Turnbull Blau.svs"]

In [4]:
fold_1 = pickle.load(open(str("inference_results_Equine-Fold-1.p"),'rb'))
fold_2 = pickle.load(open(str("inference_results_Equine-Fold-2.p"),'rb'))
fold_3 = pickle.load(open(str("inference_results_Equine-Fold-3.p"),'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'inference_results_Equine-Fold-1.p'

In [20]:
folds = {}
nms_thresh = 40
thresh = 0.35


for file_name in tqdm(human_slides):
    
    boxes1 = np.array(fold_1[file_name])
    boxes2 = np.array(fold_2[file_name])
    boxes3 = np.array(fold_3[file_name])
    
    boxes = np.concatenate([boxes1, boxes2, boxes3])
    init_shape = boxes.shape
    
    scales = ((boxes[:, 2] - boxes[:, 0]) / (boxes[:, 3] - boxes[:, 1]))
    boxes = boxes[(scales < 1.1) & (scales > 0.9)]
    
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]
    final_shape = annos.shape    
    
    #print("Input: {}  Output: {}".format(init_shape, final_shape))
    
    folds[file_name] = annos

100%|██████████| 12/12 [00:50<00:00,  2.21s/it]


In [22]:
pickle.dump(folds, open(str("InferenceAnnotationsHumaneV2.pickle"), "wb"))

In [7]:
folds = pickle.load( open( "InferenceAnnotationsHumaneV2.pickle", "rb" ) )

In [6]:
annotations_path = Path("../../Statistics/EIPH_Human_SDATA_Screening_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)

annotations.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
0,2309984,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 18095, 'x2': 18249, 'y1': 19519, 'y2': ...",e1776329-eb06-486e-bc08-1eb55c9df633,1,False,1,SDATA,Screened
1,2309988,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 16350, 'x2': 16475, 'y1': 10500, 'y2': ...",d54087bb-5634-46ba-81e7-6d9440321cf8,1,False,1,SDATA,Screened
2,2309989,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 14468, 'x2': 14618, 'y1': 22371, 'y2': ...",77c564d3-a151-4658-87d6-7bf74c74f7d0,1,False,1,SDATA,Screened
3,2309990,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 18821, 'x2': 18965, 'y1': 22047, 'y2': ...",5c537e27-4967-47e3-8a0a-5be32f0f4a6d,1,False,1,SDATA,Screened
4,2309992,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 7846, 'x2': 7990, 'y1': 20329, 'y2': 20...",ba0a81f7-8122-45b2-bde1-3980b2485a3a,1,False,1,SDATA,Screened


## find optionmal thresholds

In [8]:
def get_grade(gt, prediction):
    
    gt_mean = float(gt["grade"].mean())
    pred_mean = float(prediction[:, 4].mean())
    diff = abs(gt_mean-pred_mean)
    
    
    return gt_mean, pred_mean, diff 

In [9]:
nms_threshs = [40] 
threshs = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]

results = []
for file in tqdm(folds):
    
    dfImage = annotations[annotations["image_name"] == file] 

    for nms_thresh in nms_threshs:
        for thresh in threshs:
            
            boxes = np.array(folds[file])
            annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
            annos = annos[annos[:, 5] > thresh]
            
            gt_mean, pred_mean, diff = get_grade(dfImage, annos)
            results.append([file, nms_thresh, thresh, gt_mean, pred_mean, diff, len(dfImage), len(annos), abs(len(dfImage) - len(annos))])
            
results = pd.DataFrame(results, columns=["image_name", "nms_thresh", "thresh", "gt", "pred", "diff", "num_gt", "num_pred", "diff_num"])
results

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:24<00:00,  7.03s/it]


,image_name,nms_thresh,thresh,gt,pred,diff,num_gt,num_pred,diff_num
0,10835_19 humane BAL Turnbull Blau.svs,40,0.35,1.702889,1.730550,0.027661,11319,20553,9234
1,10835_19 humane BAL Turnbull Blau.svs,40,0.40,1.702889,1.744913,0.042024,11319,19954,8635
2,10835_19 humane BAL Turnbull Blau.svs,40,0.45,1.702889,1.761582,0.058693,11319,19319,8000
3,10835_19 humane BAL Turnbull Blau.svs,40,0.50,1.702889,1.779612,0.076723,11319,18599,7280
4,10835_19 humane BAL Turnbull Blau.svs,40,0.55,1.702889,1.801323,0.098434,11319,17687,6368
...,...,...,...,...,...,...,...,...,...
139,10672_19 humane BAL Turnbull Blau.svs,40,0.70,2.434379,2.297030,0.137349,861,1616,755
140,10672_19 humane BAL Turnbull Blau.svs,40,0.75,2.434379,2.318386,0.115993,861,1338,477
141,10672_19 humane BAL Turnbull Blau.svs,40,0.80,2.434379,2.315904,0.118474,861,918,57
142,10672_19 humane BAL Turnbull Blau.svs,40,0.85,2.434379,2.258907,0.175471,861,421,440


In [10]:
nms_threshs = [40] 
threshs = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]

optimal_th = []

for nms_thresh in nms_threshs:
    for thresh in threshs:
        
        dfTh = results[(results["nms_thresh"] == nms_thresh) & (results["thresh"] == thresh)]
        
        
        optimal_th.append([nms_thresh, thresh, dfTh["diff"].mean(), dfTh["diff_num"].mean()])
        
optimal_th = pd.DataFrame(optimal_th, columns=["nms_thresh", "thresh", "diff", "diff_num"])
optimal_th

,nms_thresh,thresh,diff,diff_num
0,40,0.35,0.201585,9865.583333
1,40,0.40,0.182198,9408.750000
2,40,0.45,0.163539,8926.750000
3,40,0.50,0.144020,8391.916667
4,40,0.55,0.133420,7772.000000
5,40,0.60,0.125267,6994.916667
6,40,0.65,0.117517,5942.666667
7,40,0.70,0.123183,4423.083333
8,40,0.75,0.147265,2922.333333
9,40,0.80,0.180414,2540.000000


In [11]:
dfTh = results[(results["nms_thresh"] == 40) & (results["thresh"] == 0.80)]
dfTh["diff_rel"] = (dfTh["diff_num"] / dfTh["num_gt"]) * 100

dfTh.sort_values("diff_rel", ascending=False)

d:\anaconda3\envs\eiph_wsi\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,image_name,nms_thresh,thresh,gt,pred,diff,num_gt,num_pred,diff_num,diff_rel
81,10227_19 humane BAL Berliner Blau.svs,40,0.8,0.736783,0.677204,0.059579,3537,11072,7535,213.033644
93,11480_19 humane BAL Berliner Blau.svs,40,0.8,1.140069,1.221050,0.080981,2306,4551,2245,97.354727
117,2702_20 BB Human BAL-001.svs,40,0.8,0.791667,0.948348,0.156681,1968,3601,1633,82.977642
9,10835_19 humane BAL Turnbull Blau.svs,40,0.8,1.702889,2.036210,0.333321,11319,4971,6348,56.082693
45,10080_19 humane BAL Berliner Blau-001.svs,40,0.8,1.465062,1.761135,0.296073,6025,2784,3241,53.792531
33,10052_19 humane BAL Berliner Blau-001.svs,40,0.8,1.807589,1.945408,0.137818,6351,3114,3237,50.968351
69,10835_19 humane BAL Berliner Blau.svs,40,0.8,1.349213,1.637792,0.288579,4576,2736,1840,40.209790
57,2707_20 TB BAL Human neu.svs,40,0.8,0.971861,1.261811,0.289950,5544,4064,1480,26.695527
21,10120_19 humane BAL Berliner Blau.svs,40,0.8,1.914302,2.035781,0.121479,8600,6400,2200,25.581395
105,11323_19 humane BAL Berliner Blau.svs,40,0.8,2.646201,2.518705,0.127496,2264,2780,516,22.791519


In [12]:
imageDict = {}

for image_name in annotations["image_name"].unique():
    
    imageDf = annotations[annotations["image_name"] == image_name]
    
    image_id = list(imageDf["image_id"])[0]
    image_set = list(imageDf["image_set"])[0]
     
    imageDict[image_name] = {"image_set": image_set, "image_id": image_id}

In [13]:
nms_thresh, thresh = 40, 0.80

inference = {}

result_annos = []
for file in tqdm(folds):
    
    image_id = imageDict[file]["image_id"]
    image_set = imageDict[file]["image_set"]
    
    boxes = np.array(folds[file])
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]
    
    for anno in annos:
        grade = anno[4]
        vector = {"x1":max(0,anno[0]), "y1": max(0, anno[1]), "x2": anno[2], "y2": anno[3]}
        
        result_annos.append([0, image_id, image_set, file, grade, vector, "/", 70, False])
    
    inference[file] = annos
  
result_annos = pd.DataFrame(result_annos, columns=["id","image_id","image_set","image_name","grade","vector","unique_identifier","user_id","deleted"])
result_annos.head()

result_annos.to_pickle("../../Statistics/EIPH_Human_SDATA_Inference_Annotations_V2.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.61it/s]
